# Mongo queries & cleaning

Este archivo contiene las consultas a la base de datos creada en Mongo, con la colección de los datos recogidos en la API open airq. Del mismo modo, la información extraída se limpia para su uso.

### Conexión con MongoDB

In [15]:
import pymongo
import pandas as pd

In [2]:
from pymongo import MongoClient

In [3]:
str_conn='mongodb://localhost:27017'  # str_conn por defecto

cursor=MongoClient(str_conn)

cursor

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
#Accede a la base de datos "ETL_project"

db = cursor.ETL_project
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ETL_project')

In [5]:
#Accede a la colección "airq"
coleccion = db.airq
coleccion

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ETL_project'), 'airq')

### Datos Madrid

Para cada ciudad, existen varias estaciones que recopilan datos de calidad de aire. Primero, se buscan todos los "id" para averiguar las estaciones por ciudad, en este caso Madrid. Posteriormente. se realiza la query para buscar el valor que queramos para todas esas estaciones.

In [113]:
#Busco todas las estaciones por "id" que hay en Madrid.

query={"city":"Madrid"}
select={"_id":0,"id":1,"city":1}

id_lst_mad = list(coleccion.find(query,select))

In [114]:
id_values = [element["id"] for element in id_lst_mad] #genera una lista con las id de todas las estación en Madrid

In [118]:
#Con todas las id de Madrid, busco el parámetro pm25 mensual.

query = {"id": {"$in": id_values}, "parameter": "pm25"}

select = {"_id": 0,"id": 1,"parameter": 1,"average": 1,"first_datetime": 1,"last_datetime": 1}

results = list(coleccion.find(query, select))

In [119]:
mad25 = pd.DataFrame(results)

In [126]:
mad25.head(20)

,id,average,parameter,first_datetime,last_datetime
0,3265,13.607143,pm25,2022-01-01T01:00:00Z,2022-02-01T00:00:00Z
1,3265,11.491049,pm25,2022-02-01T04:00:00Z,2022-02-28T23:00:00Z
2,3265,11.305389,pm25,2022-03-01T01:00:00Z,2022-04-01T00:00:00Z
3,3265,5.217391,pm25,2022-04-01T01:00:00Z,2022-04-30T07:00:00Z
4,3265,9.842593,pm25,2022-05-01T01:00:00Z,2022-05-31T23:00:00Z
5,3265,9.505747,pm25,2022-06-01T01:00:00Z,2022-06-30T02:00:00Z
6,3265,12.936709,pm25,2022-07-01T01:00:00Z,2022-07-30T05:00:00Z
7,3265,11.609756,pm25,2022-08-01T01:00:00Z,2022-08-31T23:00:00Z
8,3265,7.134021,pm25,2022-09-01T01:00:00Z,2022-10-01T00:00:00Z
9,3265,16.056075,pm25,2022-10-01T01:00:00Z,2022-10-28T02:00:00Z


In [121]:
mad25.shape

(767, 5)

Este data frame contiene el dato de media mensual de pm2.5 de 2022 para cada una de las estaciones disponibles en Madrid. Quiero un valor medio único para madrid, por tanto agrupo por mes y hago la media de cada conjunto de valores, usando **group by y mean() como función de agregación.**

La materia particulada o PM (por sus siglas en inglés) 2.5, son partículas muy pequeñas en el aire que tiene un diámetro de 2.5 micrómetros (aproximadamente 1 diezmilésimo de pulgada) o menos de diámetro. Una evaluación de la organización, realizada en 2013, determinó que la contaminación del aire por partículas PM 2.5 es carcinógena para el ser humano y está estrechamente relacionadas con la incidencia al cáncer de pulmón.

In [139]:
mad25_avg = mad25.groupby("first_datetime")["average"].mean()
mad25_avg

first_datetime
2022-01-01T01:00:00Z    11.194263
2022-02-01T04:00:00Z    10.523692
2022-02-07T00:00:00Z     8.462633
2022-03-01T01:00:00Z    11.375703
2022-03-01T10:00:00Z    11.896024
2022-04-01T01:00:00Z     6.955307
2022-04-02T00:00:00Z     6.065217
2022-04-02T03:00:00Z     5.577204
2022-04-02T04:00:00Z     5.670189
2022-05-01T01:00:00Z     9.555705
2022-05-03T00:00:00Z     9.480442
2022-05-05T04:00:00Z     9.497609
2022-05-07T04:00:00Z     8.534884
2022-06-01T01:00:00Z    10.262554
2022-06-01T14:00:00Z    11.373494
2022-06-02T23:00:00Z     9.714758
2022-07-01T01:00:00Z    12.462989
2022-07-01T20:00:00Z    18.547019
2022-08-01T01:00:00Z    11.328731
2022-08-01T09:00:00Z    13.998792
2022-09-01T01:00:00Z     8.112390
2022-09-07T01:00:00Z     5.723861
2022-10-01T01:00:00Z    11.657261
2022-11-02T06:00:00Z     7.522817
2022-11-02T13:00:00Z     8.382696
2022-12-01T01:00:00Z     9.462778
Name: average, dtype: float64

In [144]:
df_mad25_avg = mad25_avg.to_frame() #lo convierto en df.

Se debería realizar el mismo proceso para cada ciudad. Por comodidad, para el resto de ciudades se ha tomado el valor de una única estación.

### Datos Málaga

In [145]:
#El nombre de una estación en Málaga es "ES0817A". Este nombre lo puedo consultar en la bd en Mongo, lo he extraído de la API.

query={"name":"ES0817A",
       "parameter":"pm25"}
select={"_id":0,"name":1,"parameter":1,"average":1, "first_datetime":1, "last_datetime":1}

results2= list(coleccion.find(query,select))

In [146]:
df_malaga = pd.DataFrame(results2)

In [148]:
df_malaga.head()

,name,average,parameter,first_datetime,last_datetime
0,ES0817A,13.405634,pm25,2022-01-01T01:00:00Z,2022-02-01T00:00:00Z
1,ES0817A,12.812689,pm25,2022-02-01T04:00:00Z,2022-02-28T20:00:00Z
2,ES0817A,14.445460,pm25,2022-03-01T08:00:00Z,2022-04-01T00:00:00Z
3,ES0817A,5.960000,pm25,2022-04-01T01:00:00Z,2022-04-30T10:00:00Z
4,ES0817A,9.836176,pm25,2022-05-03T01:00:00Z,2022-05-31T16:00:00Z


In [161]:
df_malaga.shape #chachi, tiene los doce meses de 2022

(12, 5)

### Datos Barcelona

In [156]:
#id: 276005, "name" = "Starlab Barcelona"

query={"name":"Starlab Barcelona",
       "parameter":"pm25"}
select={"_id":0,"name":1,"parameter":1,"average":1, "first_datetime":1, "last_datetime":1}

results3= list(coleccion.find(query,select))

In [157]:
df_bcn = pd.DataFrame(results3)

In [158]:
df_bcn.head()

,name,average,parameter,first_datetime,last_datetime
0,Starlab Barcelona,28.570915,pm25,2022-01-07T10:00:00Z,2022-02-01T00:00:00Z
1,Starlab Barcelona,13.440817,pm25,2022-02-01T01:00:00Z,2022-02-25T11:00:00Z
2,Starlab Barcelona,16.697877,pm25,2022-03-03T14:00:00Z,2022-04-01T00:00:00Z
3,Starlab Barcelona,8.251961,pm25,2022-04-01T01:00:00Z,2022-05-01T00:00:00Z
4,Starlab Barcelona,11.750848,pm25,2022-05-01T01:00:00Z,2022-06-01T00:00:00Z


In [160]:
df_bcn.shape 

(12, 5)

### Datos Sevilla

In [162]:
#id: 4219,"name" = "ES0817A".

query={"name":"ES0817A",
       "parameter":"pm25"}
select={"_id":0,"name":1,"parameter":1,"average":1, "first_datetime":1, "last_datetime":1}

results4= list(coleccion.find(query,select))

In [163]:
df_sev = pd.DataFrame(results4)

In [164]:
df_sev.head()

,name,average,parameter,first_datetime,last_datetime
0,ES0817A,13.405634,pm25,2022-01-01T01:00:00Z,2022-02-01T00:00:00Z
1,ES0817A,12.812689,pm25,2022-02-01T04:00:00Z,2022-02-28T20:00:00Z
2,ES0817A,14.445460,pm25,2022-03-01T08:00:00Z,2022-04-01T00:00:00Z
3,ES0817A,5.960000,pm25,2022-04-01T01:00:00Z,2022-04-30T10:00:00Z
4,ES0817A,9.836176,pm25,2022-05-03T01:00:00Z,2022-05-31T16:00:00Z


In [165]:
df_sev.shape

(12, 5)

### Datos Valencia

In [166]:
#id: 4316 "name" = "ES1181A" 

query={"name":"ES1181A",
       "parameter":"pm25"}
select={"_id":0,"name":1,"parameter":1,"average":1, "first_datetime":1, "last_datetime":1}

results5= list(coleccion.find(query,select))

In [168]:
df_val = pd.DataFrame(results5)

In [169]:
df_val.head()

,name,average,parameter,first_datetime,last_datetime
0,ES1181A,10.937768,pm25,2022-01-01T01:00:00Z,2022-01-31T20:00:00Z
1,ES1181A,12.289189,pm25,2022-02-01T04:00:00Z,2022-03-01T00:00:00Z
2,ES1181A,11.417417,pm25,2022-03-01T01:00:00Z,2022-04-01T00:00:00Z
3,ES1181A,7.034884,pm25,2022-04-01T01:00:00Z,2022-05-01T00:00:00Z
4,ES1181A,13.381579,pm25,2022-05-02T00:00:00Z,2022-06-01T00:00:00Z


In [170]:
df_val.shape

(12, 5)

### Datos Bilbao

In [171]:
#id: 3264, "name" = "ES1489A"

query={"name":"ES1489A",
       "parameter":"pm25"}
select={"_id":0,"name":1,"parameter":1,"average":1, "first_datetime":1, "last_datetime":1}

results6= list(coleccion.find(query,select))

In [172]:
df_bilbo = pd.DataFrame(results6)

In [173]:
df_bilbo.head()

,name,average,parameter,first_datetime,last_datetime
0,ES1489A,13.695238,pm25,2022-03-13T01:00:00Z,2022-04-01T00:00:00Z
1,ES1489A,3.459184,pm25,2022-04-01T01:00:00Z,2022-04-30T06:00:00Z
2,ES1489A,5.811475,pm25,2022-05-01T01:00:00Z,2022-05-31T04:00:00Z
3,ES1489A,6.168539,pm25,2022-06-01T01:00:00Z,2022-06-30T02:00:00Z
4,ES1489A,8.663158,pm25,2022-07-01T01:00:00Z,2022-07-30T04:00:00Z


In [174]:
df_bilbo.shape #en Bilbao no están los primeros meses de 2022

(10, 5)